# Machine Learning Study Notes

This post contains notes of building a neural network with Softmax loss and Hinge loss.

This post is one of an ongoing series on machine learning studies:
1. [Pytorch]({% post_url 2021-05-04-machine-learning-pytorch %})
2. [Loss function(this)]({% post_url 2021-05-07-machine-learning-loss %}): Hinge Loss, Softmax
3. [Backpropagation]({% post_url 2021-05-07-machine-learning-backpropagation %})